In [ ]:
import pandas as pd
import numpy as np

# other .py file
import data_utils as du
import Model_selection as Ms
import Plot_utils as pu
import matplotlib.pyplot as plt

from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold



# setting path 
pkl_path = "D:\ICMLA\Dreamer\Dreamer_Raw(normalized)\Denoised.pkl"
pkl_path = pkl_path.replace('\\', '/')
model_save_path = 'D:/Model/'
model_name = 'Fusion_model_Attention_7Convlayer_'


# setting 
np.random.seed(12)
sampling_rate = 256
left = int(np.round(0.04*6*256)) # 61
right = int(np.round(0.04*10*256)) # 102
window = 256 * 5
accuracies =[]
fold_number = 1






# Data Load 
excite_b, neutral_b, stress_b = du.load_DREAMER_class(pkl_path, 'beat', window)
excite_r, neutral_r, stress_r = du.load_DREAMER_class(pkl_path, 'rhythm', window)

print(excite_b.shape, neutral_b.shape, stress_b.shape)
print(excite_r.shape, neutral_r.shape, stress_r.shape)

excite_b = excite_b[:excite_r.shape[0]]
neutral_b = neutral_b[:neutral_r.shape[0]]
stress_b = stress_b[:stress_r.shape[0]]

print(excite_b.shape, neutral_b.shape, stress_b.shape)
print(excite_r.shape, neutral_r.shape, stress_r.shape)

# labeling
excite_b_label, excite_r_label = np.zeros(excite_b.shape[0], np.int32), np.zeros(excite_r.shape[0], np.int32)
neutral_b_label, neutral_r_label = np.ones(neutral_b.shape[0], np.int32), np.ones(neutral_r.shape[0], np.int32)
stress_b_label, stress_r_label = 2*np.ones(stress_b.shape[0], np.int32), np.ones(stress_r.shape[0], np.int32)

print(excite_b_label.shape, neutral_b_label.shape, stress_b_label.shape)
print(excite_r_label.shape, neutral_r_label.shape, stress_r_label.shape)

# data concatenate
Train_beat = np.concatenate([excite_b, neutral_b, stress_b])
Train_rhythm = np.concatenate([excite_r, neutral_r, stress_r])
print(Train_beat.shape, Train_rhythm.shape)

Train_beat_label = np.concatenate([excite_b_label, neutral_b_label, stress_b_label])
Train_rhythm_label = np.concatenate([excite_r_label, neutral_r_label, stress_r_label])

print(Train_beat_label.shape, Train_rhythm_label.shape)

train_beat, train_rhythm, train_y = du.shuffle_data2(Train_beat, Train_rhythm, Train_beat_label)

train_label = to_categorical(train_y)







# 10-Fold Cross validation

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in skf.split(train_beat, train_y):
    X_train_b, X_test_b, X_train_r, X_test_r = train_beat[train_index], train_beat[test_index], train_rhythm[train_index], train_rhythm[test_index]
    Y_train, Y_test = train_y[train_index], train_y[test_index]
    
    y_train = to_categorical(Y_train)
    y_test = to_categorical(Y_test)
    
    print(X_train_b.shape, X_test_b.shape, X_train_r.shape, X_test_r.shape)
    print(Y_train.shape, Y_test.shape)
    
    model = Ms.fusion_model(X_train_b.shape[1], X_train_r.shape[1])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit([X_train_b, X_train_r], y_train, validation_data=([X_test_b, X_test_r],y_test), epochs=60, batch_size=32,) 
    
    # 평가    
    scores = model.evaluate([X_test_b,X_test_r], y_test)
    pu.Plot_lr_curve(history)
    model.save(model_save_path + model_name + str(fold_number) + '(' + str(np.round(scores[1]*100,3)) + ').h5')
    pred = model.predict([X_test_b,X_test_r ])
    pu.Plot_confusion_matrix(y_test, pred)
    
    accuracies.append(scores[1])
    
average_accuracy = np.mean(accuracies)
print('평균 정확도 : ', average_accuracy)